In [11]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
# System imports
import os
import sys
import yaml

# External imports
import scipy as sp
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

sys.path.append('..')
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import nersc_tensorboard_helper

In [13]:
from LightningModules.GNN.Models.ecgnn import ECGNN

In [20]:
with open("example_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [21]:
model = ECGNN(hparams)
logger = TensorBoardLogger("tb_ecgnn_logs",name = "ecgnn")
trainer = Trainer(gpus=1,max_epochs=20,logger=logger)
torch.autograd.set_detect_anomaly(True)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name           | Type        | Params
-----------------------------------------------
0 | input_network  | Sequential  | 384   
1 | node_network   | NodeNetwork | 17.2 K
2 | edge_network   | EdgeNetwork | 18.6 K
3 | output_network | Sequential  | 12.7 K
-----------------------------------------------
49.0 K    Trainable params
0         Non-trainable params
49.0 K    Total params


Epoch 0:  50%|█████     | 20/40 [00:29<00:29,  1.49s/it, loss=670, v_num=197]     
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 20/40 [00:29<00:29,  1.50s/it, loss=120, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 20/40 [00:30<00:30,  1.51s/it, loss=124, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 20/40 [00:30<00:30,  1.52s/it, loss=123, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 20/40 [00:30<00:30,  1.53s/it, loss=116, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 20/40 [00:30<00:30,  1.54s/it, loss=59, v_num=197]  
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 20/40 [00:30<00:30,  1.53s/it, loss=44.2, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 20/40 [00:30<00:30,  1.53s/it, loss=33.2, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 20/40 [00:30<00:30,  1.54s/it, loss=36.6, v_num=197]
Validating: 0it [00:00, ?it/s]
Epoch 9:  

1

In [16]:
test_results = trainer.test(ckpt_path=None)
print(test_results)

Set SLURM handle signals.


Testing: 100%|██████████| 20/20 [00:07<00:00,  2.50it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'current_lr': 0.0023999998811632395,
 'eff': 0.6067168712615967,
 'loss': 26.3963680267334,
 'pur': 0.615739107131958,
 'ratio': 0.21849309902707595,
 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32),
 'val_loss': 27.093305587768555}
--------------------------------------------------------------------------------
[{'loss': 26.3963680267334, 'eff': 0.6067168712615967, 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32), 'ratio': 0.21849309902707595, 'val_loss': 27.093305587768555, 'pur': 0.615739107131958, 'current_lr': 0.0023999998811632395}]


In [17]:
os.environ['TENSORBOARD_BINARY'] = "/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/bin/tensorboard"
%tensorboard --logdir tb_ecgnn_logs/ --port 0
nersc_tensorboard_helper.tb_address()


Reusing TensorBoard on port 39007 (pid 53849), started 0:50:41 ago. (Use '!kill 53849' to kill it.)

In [11]:
!ps aux | grep tensorboard

jferguso 16183 21.9  0.0  14136  3556 pts/2    Ss+  15:53   0:00 /bin/bash -c ps aux | grep tensorboard
jferguso 16193  0.0  0.0   8984   900 pts/2    S+   15:53   0:00 grep tensorboard
